In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
# Any results you write to the current directory are saved as output.

/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv
/kaggle/input/pubg-finish-placement-prediction/train_V2.csv
/kaggle/input/pubg-finish-placement-prediction/test_V2.csv


In [2]:
df_train_raw=pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')
#read data_set from kernel

In [3]:
df_train_raw=df_train_raw[df_train_raw['kills']<=30]
#clean the data,we assume the kill>30 are cheating in the game

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import sklearn
#import some lib
df=df_train_raw.drop(columns=['Id','groupId','matchId','matchType'])
#drop some rows that are not going to be used in the training
df=df.sample(n=100000)
#choose 100,000 columns randomly to train our model
#for we do not have plenty of time to train our model in GPU, we just select a small batch of the data

In [5]:
df = df.values
df = np.array(df)
#read the dataform to array
print(df)

[[0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 6.250e-02]
 [0.000e+00 4.000e+00 3.031e+02 ... 3.000e+00 1.470e+03 8.571e-01]
 [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 2.170e-02]
 ...
 [0.000e+00 0.000e+00 1.000e+02 ... 1.000e+00 1.481e+03 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.000e+00 0.000e+00 1.778e-01]
 [0.000e+00 0.000e+00 4.680e+01 ... 5.000e+00 0.000e+00 8.571e-01]]


In [6]:
for i in range(23):
    df[:,i] = (df[:,i]-df[:,i].min())/(df[:,i].max()-df[:,i].min())
#Data normalization 

In [7]:
x_data = df[:,:23]
y_data = df[:,23]

In [8]:
#build the train model
x = tf.placeholder(tf.float32,[None,23],name = "X")          #24 input 
y = tf.placeholder(tf.float32,[None,1],name = "Y")           # 1 output
Weights_L1 = tf.Variable(tf.random_normal([23,20],stddev=0.01))
biases_L1 = tf.Variable(tf.zeros([1,20]))
Wx_plus_b_L1 = tf.matmul(x,Weights_L1) + biases_L1
L1 = tf.tanh(Wx_plus_b_L1)  #activiation founction
w = tf.Variable(tf.random_normal([20,1])) #hidden layer
b = tf.Variable(tf.zeros([1,20]))
Wx_plus_b_L2 = tf.matmul(L1,w) + b
pred = tf.nn.relu(Wx_plus_b_L2)

In [9]:
from sklearn.utils import shuffle
train_epochs = 1
learning_rate = 0.01
with tf.name_scope("LossFunction"):
    loss_function = tf.reduce_mean(tf.pow(y-pred,2))    
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

sess = tf.Session()

#init variable
init = tf.global_variables_initializer()

#run session
sess.run(init)

#Iterative training
for epoch in range(train_epochs):
    loss_sum = 0.0
    for xs,ys in zip(x_data,y_data):
        
        xs = xs.reshape(1,23)
        ys = ys.reshape(1,1)
        _,loss = sess.run([optimizer,loss_function],feed_dict={x:xs,y:ys})

    x_data,y_data = shuffle(x_data,y_data)

    b0temp = b.eval(session=sess)            
    w0temp = w.eval(session=sess)  

    loss_average = loss_sum/len(y_data) 
    
    print("epoch=",epoch+1,"loss=",loss_average,"b=",b0temp,"w=",w0temp)
#for fastest training, i just set the epoch to 1. But it should be set above 10 to auarantee the accuracy

epoch= 1 loss= 0.0 b= [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] w= [[ 1.0343236 ]
 [-0.23345436]
 [-0.66320235]
 [-0.5057406 ]
 [-1.4873843 ]
 [-1.2115022 ]
 [ 1.1697094 ]
 [-0.03152117]
 [-1.1380386 ]
 [ 1.8179114 ]
 [-1.650698  ]
 [-0.9362165 ]
 [ 0.5144368 ]
 [ 0.12752476]
 [ 2.236437  ]
 [ 1.1018323 ]
 [ 0.20238817]
 [-1.060732  ]
 [ 0.90985465]
 [-0.57679427]]


In [10]:
df_test1_raw=pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')
#load the test data

In [11]:
df_test=df_test1_raw.drop(columns=['Id','groupId','matchId','matchType'])
df_test = df_test.values
df_test = np.array(df_test)
for i in range(23):
    df_test[:,i] = (df_test[:,i]-df_test[:,i].min())/(df_test[:,i].max()-df_test[:,i].min())
x__data_test = df_test[:,:23]
y__data_test = df_test[:,23]

In [12]:
#calculate the prediction with the model trained before
row=len(df_test)
pre=np.zeros(row)
#create a zero array to save the prediction value
for n in range(1,row):
    x_test = x__data_test[n]
    x_test
    x_test = x_test.reshape(1,23)
    predict = sess.run(pred,feed_dict={x:x_test})
    pre[n]=predict[:,1]
print(pre)

[0. 0. 0. ... 0. 0. 0.]


In [13]:
df_submit=pd.read_csv("../input/pubg-finish-placement-prediction/sample_submission_V2.csv")
#load the submission file from kaggle's file folder

In [14]:
df_submit['winPlacePerc']=pre
#replace the 'winPlacePrec' with prediction value

In [15]:
print(df_submit)
#check the format of the file

                     Id  winPlacePerc
0        9329eb41e215eb           0.0
1        639bd0dcd7bda8           0.0
2        63d5c8ef8dfe91           0.0
3        cf5b81422591d1           0.0
4        ee6a295187ba21           0.0
...                 ...           ...
1934169  a316c3a13887d5           0.0
1934170  5312146b27d875           0.0
1934171  fc8818b5b32ad3           0.0
1934172  a0f91e35f8458f           0.0
1934173  3696fc9f3a42b2           0.0

[1934174 rows x 2 columns]


In [16]:
df_submit.to_csv('../working/submission.csv',index=None)
#save the file to specified path